In [1]:
from SKLR.SKLR import SKLR
import torch
import numpy as np
import time
import scipy

from scipy.spatial.distance import pdist, squareform
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.metrics.pairwise import pairwise_kernels


#import importlib
#importlib.reload(some_module)




In [2]:
n=2048
d=10
X0=np.random.rand(n//2,d)
X1=np.random.rand(n//2,d)
X=np.vstack([X0,-X1-1])
Y0=np.ones(n//2)
Y1=np.ones(n//2)
Y=np.append(-Y0,Y1)
model_SKLR=SKLR(X,Y)
model_SKLR.fit()

[9.99996079e-01 9.99980476e-01 9.99996639e-01 ... 5.40109412e-04
 9.06442064e-04 5.00926819e-04]
[0.99988318 0.9985034  0.99999684 ... 1.         1.         1.        ]
[0.81663559 0.81377498 0.81206078 ... 0.50237356 0.50556088 0.50181618]
[0.55344726 0.55641288 0.54629121 ... 0.87395293 0.8734361  0.87883032]
[0.16165912 0.17132013 0.15558582 ... 0.95299157 0.952237   0.95658833]
[4.19882958e-04 5.68019052e-04 3.56027755e-04 ... 9.80588073e-01
 9.79189650e-01 9.83457368e-01]
[0.         0.         0.         ... 0.99617502 0.61547543 0.99992381]
[0.         0.         0.         ... 0.99995779 0.96510655 0.99999974]
[0.         0.         0.         ... 0.99999769 0.99624362 0.99999999]
[0.         0.         0.         ... 0.99999917 0.99854469 1.        ]
[0.         0.         0.         ... 0.99999928 0.99871276 1.        ]
[0.         0.         0.         ... 0.99999928 0.998716   1.        ]
[0.         0.         0.         ... 0.99999928 0.998716   1.        ]


In [3]:
model_SKLR.probability

array([0.        , 0.        , 0.        , ..., 0.99999928, 0.998716  ,
       1.        ])

In [5]:
model_SKLR.accuracy

0.9853515625

In [6]:
model_SKLR.iteration

13

In [6]:
model_SKLR.time

0.057700157165527344

In [ ]:

d=2
for i in range(5,13):
    n=2**i
    X0=np.random.rand(n//2,d)
    X1=np.random.rand(n//2,d)
    X=np.vstack([X0,-X1-1])
    Y0=np.ones(n//2)
    Y1=np.ones(n//2)
    Y=np.append(-Y0,Y1)
    model_SKLR=SKLR(X,Y,kernel="rbf")
    model_SKLR.fit()
    print(model_SKLR.time)

0.0014040470123291016
0.0024602413177490234
0.0027298927307128906
0.0010387897491455078
0.0017752647399902344
0.06200098991394043
6.35811710357666


In [ ]:
matrix_rank

In [2]:
n=50
A=np.random.rand(n,n)

time_start=time.time()
scipy.linalg.pinv(A)
time_end=time.time()
print(time_end-time_start)


time_start=time.time()
scipy.linalg.inv(A)
time_end=time.time()
print(time_end-time_start)

0.00440216064453125
0.0021152496337890625


In [9]:
np.linalg.matrix_rank(A)

50